In [ ]:
import numpy as np
import pandas as pd
import requests
import pickle
from bs4 import BeautifulSoup
import jieba
import re
import itertools
import time
import os
import json

In [ ]:
# scrapy everything down
cywl_list = ["https://guba.eastmoney.com/list,603569_" + str(i) + ".html" for i in range(1,370)]
cywl_iterator = iter(cywl_list)
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0)Gecko/20100101 Firefox/87.0"}
results = []
count = 1
for url in cywl_iterator:
    print(url)
    ind_not_get_data = True
    try:
        res = requests.get(url,headers = headers,timeout=10)
        time.sleep(np.random.randint(0,3))
        ind_not_get_data = False
        #res.encoding("GB2312")
        print(f"{count}page is done")
        results.append(res)
        count += 1
    except:
        print("Fail because of Timeout")
        url_temp = url

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")

for i in range(len(results)):
    name = str(i) + ".html"
    with open("data/" + name,"w+",encoding="utf-8") as f:
        res = results[i]
        res.encoding=res.apparent_encoding
        f.write(results[i].text)

In [ ]:
# how I get the right RE
""" 
p1 = r'<tr.+?ter">.+?(\d{4}-\d{2}-\d{2}).+?</div.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div>.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div>'
p2 = r'<tr data-index.+?<td class="decs_col".+?<a href=.+?>(.+?)<.+?'
pattern2 = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">(.+?)</a>.+?"author">(.+?)</a>.+?"update">(.+?)</div>'
pattern3 = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">.+?.html">(.+?)</a>.+?"author">(.+?)</a>.+?"update">(.+?)</div>'
pattern4 = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">.+?.html">(.+?)</a>.+?"author">.+?">(.+?)</a>.+?"update">(.+?)</div>'
"""
p = r'class="listitem">.+?"read">(.+?)</div>.+?"reply">(.+?)</div>.+?"title">.+?.html">(.+?)</a>.+?"author">.+?">(.+?)</a>.+?"update">(.+?)</div>'

with open("10.html","r",encoding = "utf-8") as f:
    res = f.read()
match = re.findall(p,res,re.S)
match

In [ ]:
if not os.path.exists("res"):
    os.makedirs("res")

results = []
num = []
count = 0
for i in range(361):
    name = str(i) + ".html"
    with open(name,"r",encoding="utf-8") as f:
        temp = f.read()
    result = re.findall(p,temp,re.S)
    results += result
res_df = pd.DataFrame(results,columns=["comment","reply","content","author","time"])
#res_df.to_excel("result/cywl_gb.xlsx",encoding="utf-8")
year = 2023
for i in range(len(res_df["time"])-1):
    if res_df["time"][i][:2] == "01" and res_df["time"][i+1][:2] == "12":
        year -= 1
    res_df["time"][i] = str(year) + "-" + res_df["time"][i]
res_df.iloc[-1,4] = str(year) + "-" + res_df.iloc[-1,4]
res_df["time"] = pd.to_datetime(res_df["time"])
res_df["time2"] = pd.to_datetime(res_df["time"].apply(lambda x:str(x)[:10]),format = "%Y-%m-%d")
i1 = res_df["time"].groupby(res_df["time2"]).count()
i2 = res_df["reply"].groupby(res_df["time2"]).mean()
i3 = pd.Series(i1 * i2,name="flc")
res_df.to_excel("res/res.xlsx",encoding="utf-8")